In [1]:
''' Basics of Flask '''

' Basics of Flask '

In [11]:
# 1. Create a Flask app that displays "Hello, World!" on the homepage.
'''
Save this as app.py.
Open terminal in the same directory.
Run: python app.py
Visit http://127.0.0.1:5000/ in your browser.
'''

from flask import Flask

# Set up the Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, World!"

# Use Flask's test client
with app.test_client() as client:
    response = client.get('/')
    print("Response:", response.data.decode())



Response: Hello, World!


In [7]:
pip install flask

In [17]:
# 2. Build a Flask app with static HTML pages and navigate between them.

'''
How to Run:
Save all files in a folder named flask_app_2.
In terminal, navigate to the folder and run:
python app.py
Visit http://127.0.0.1:5000/ for the home page and use the link to navigate to the about page.
'''

# app.py
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

if __name__ == '__main__':
    app.run(debug=True)


# templates/home.html
<!DOCTYPE html>
<html>
<head>
    <title>Home</title>
</head>
<body>
    <h1>Welcome to the Home Page!</h1>
    <a href="/about">Go to About Page</a>
</body>
</html>


# templates/about.html

<!DOCTYPE html>
<html>
<head>
    <title>About</title>
</head>
<body>
    <h1>This is the About Page!</h1>
    <a href="/">Back to Home</a>
</body>
</html>



SyntaxError: invalid syntax (2893635540.py, line 29)

In [19]:
# 3. Develop a Flask app that uses URL parameters to display dynamic content.

'''
How to Run:
Save this as app.py inside a folder named flask_app_3.
Run it with python app.py.
Visit URLs like:
http://127.0.0.1:5000/hello/john
http://127.0.0.1:5000/hello/sarah
'''

# app.py

from flask import Flask

app = Flask(__name__)

@app.route('/hello/<username>')
def greet_user(username):
    return f"<h1>Hello, {username.title()}!</h1>"

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [ ]:
# 4. Create a Flask app with a form that accepts user input and displays it.

'''
How to Run:
Save all files in a folder called flask_app_4.
Run: python app.py
Open http://127.0.0.1:5000/ in a browser.
Enter a name in the form, submit it, and see it displayed.
'''

# app.py
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        name = request.form['name']
        return render_template('result.html', name=name)
    return render_template('form.html')

if __name__ == '__main__':
    app.run(debug=True)


# templates/form.html
<!DOCTYPE html>
<html>
<head>
    <title>User Form</title>
</head>
<body>
    <h2>Enter your name:</h2>
    <form method="POST">
        <input type="text" name="name" required>
        <input type="submit" value="Submit">
    </form>
</body>
</html>


# templates/result.html

<!DOCTYPE html>
<html>
<head>
    <title>Result</title>
</head>
<body>
    <h2>Hello, {{ name }}!</h2>
    <a href="/">Go back</a>
</body>
</html>


In [ ]:
# 5. Implement user sessions in a Flask app to store and display user-specific data.
'''
How to Run:
Save these in a folder named flask_app_5.
Run: python app.py
Go to http://127.0.0.1:5000/
Enter your name — it will be stored in a session and shown on the welcome page.
'''

# app.py
from flask import Flask, render_template, request, session, redirect, url_for

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'  # Required to use sessions

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        username = request.form['username']
        session['username'] = username
        return redirect(url_for('welcome'))
    return render_template('index.html')

@app.route('/welcome')
def welcome():
    if 'username' in session:
        return render_template('welcome.html', username=session['username'])
    return redirect(url_for('index'))

@app.route('/logout')
def logout():
    session.pop('username', None)
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)


# templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>Login Page</title>
</head>
<body>
    <h2>Enter your name:</h2>
    <form method="POST">
        <input type="text" name="username" required>
        <input type="submit" value="Login">
    </form>
</body>
</html>


# templates/welcome.html
<!DOCTYPE html>
<html>
<head>
    <title>Welcome</title>
</head>
<body>
    <h2>Welcome, {{ username }}!</h2>
    <a href="/logout">Logout</a>
</body>
</html>





In [21]:
''' Intermediate Flask Topics '''


' Intermediate Flask Topics '

In [ ]:
# 6. Build a Flask app that allows users to upload files and display them on the website.

'''
Flask App Structure
app.py - Main Flask application
templates/ - Folder for HTML templates
upload.html - Upload form and display area
uploads/ - Folder where uploaded files are stored
'''

# 1. Flask Backend (app.py)

import os
from flask import Flask, render_template, request, redirect, url_for, send_from_directory
from werkzeug.utils import secure_filename

app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure the upload folder exists
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return 'No file part'
        file = request.files['file']
        if file.filename == '':
            return 'No selected file'
        if file:
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return redirect(url_for('uploaded_file', filename=filename))
    return render_template('upload.html')

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return render_template('upload.html', filename=filename)

@app.route('/display/<filename>')
def display_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__ == '__main__':
    app.run(debug=True)

'''
# 2. HTML Template (templates/upload.html)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Upload and Display File</title>
</head>
<body>
    <h1>Upload a File</h1>
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="file" required>
        <input type="submit" value="Upload">
    </form>

    {% if filename %}
        <h2>Uploaded File:</h2>
        <!-- For images -->
        <img src="{{ url_for('display_file', filename=filename) }}" alt="Uploaded File" style="max-width:400px;">
        <!-- For other files, you can provide a download link -->
        <p><a href="{{ url_for('display_file', filename=filename) }}">Download/View File</a></p>
    {% endif %}
</body>
</html>


3. How It Works
User visits the site and sees the upload form.
After selecting and submitting a file, the file is saved to the uploads directory.
The page then displays the uploaded file using the /display/<filename> route, which serves the file from the server.
The template displays the image if it's an image file, and provides a download link for any file type.

'''

In [ ]:
# 7. Integrate a SQLite database with Flask to perform CRUD operations on a list of items.

# 1. Setup and Configuration
# Install dependencies:

pip install flask flask-sqlalchemy

# File structure:
project/
├── app.py
├── database.db
└── templates/
    ├── index.html
    ├── add_item.html
    └── edit_item.html
# 2. Database Model (app.py)

from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

class Item(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    description = db.Column(db.String(200))

# Create tables before first request
@app.before_first_request
def create_tables():
    db.create_all()

# 3. CRUD Operations
# Create (POST)

@app.route('/add', methods=['GET', 'POST'])
def add_item():
    if request.method == 'POST':
        new_item = Item(
            name=request.form['name'],
            description=request.form['description']
        )
        db.session.add(new_item)
        db.session.commit()
        return redirect(url_for('index'))
    return render_template('add_item.html')

# Read (GET)

@app.route('/')
def index():
    items = Item.query.all()
    return render_template('index.html', items=items)

# Update (PUT)

@app.route('/edit/<int:id>', methods=['GET', 'POST'])
def edit_item(id):
    item = Item.query.get_or_404(id)
    if request.method == 'POST':
        item.name = request.form['name']
        item.description = request.form['description']
        db.session.commit()
        return redirect(url_for('index'))
    return render_template('edit_item.html', item=item)

# Delete (DELETE)

@app.route('/delete/<int:id>')
def delete_item(id):
    item = Item.query.get_or_404(id)
    db.session.delete(item)
    db.session.commit()
    return redirect(url_for('index'))

'''
4. HTML Templates
index.html (List all items):
<!DOCTYPE html>
<html>
<body>
    <h1>Items</h1>
    <a href="{{ url_for('add_item') }}">Add New</a>
    <ul>
        {% for item in items %}
            <li>
                {{ item.name }} - {{ item.description }}
                <a href="{{ url_for('edit_item', id=item.id) }}">Edit</a>
                <a href="{{ url_for('delete_item', id=item.id) }}">Delete</a>
            </li>
        {% endfor %}
    </ul>
</body>
</html>

add_item.html (Create form):
<form method="POST">
    <input type="text" name="name" placeholder="Name" required>
    <input type="text" name="description" placeholder="Description">
    <button type="submit">Add Item</button>
</form>
'''

# 5. Run the Application
if __name__ == '__main__':
    app.run(debug=True)


'''
Key Features
SQLAlchemy ORM: Manages database interactions using Python objects
Automatic Table Creation: db.create_all() initializes the database
Error Handling: get_or_404() provides automatic 404 responses
Form Handling: Simple HTML forms for user input

To use the application:
Run python app.py
Visit http://localhost:5000
Add items using the form
Edit/Delete existing items through the interface

For production, consider:
Using PostgreSQL/MySQL instead of SQLite
Adding form validation
Implementing user authentication
Using Flask-WTF for secure forms
'''


In [ ]:
# 8. Implement user authentication and registration in a Flask app using Flask-Login.

# 1. Install Required Packages

pip install flask flask-login flask-sqlalchemy werkzeug

# 2. Project Structure

your_app/
├── app.py
├── models.py
├── db.sqlite
└── templates/
    ├── base.html
    ├── index.html
    ├── login.html
    ├── signup.html
    └── profile.html


# 3. User Model
# models.py

from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin

db = SQLAlchemy()

class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(100), unique=True)
    name = db.Column(db.String(1000))
    password = db.Column(db.String(100))

# 4. App and Login Manager Initialization

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager

db = SQLAlchemy()

def create_app():
    app = Flask(__name__)
    app.config['SECRET_KEY'] = 'your-secret-key'
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite'
    db.init_app(app)

    from .models import User

    login_manager = LoginManager()
    login_manager.login_view = 'auth.login'
    login_manager.init_app(app)

    @login_manager.user_loader
    def load_user(user_id):
        return User.query.get(int(user_id))

    # Register blueprints for auth and main routes
    from .auth import auth as auth_blueprint
    app.register_blueprint(auth_blueprint)

    from .main import main as main_blueprint
    app.register_blueprint(main_blueprint)

    return app

# 5. Authentication Routes

from flask import Blueprint, render_template, redirect, url_for, request, flash
from werkzeug.security import generate_password_hash, check_password_hash
from flask_login import login_user, logout_user, login_required
from .models import User, db

auth = Blueprint('auth', __name__)

@auth.route('/login')
def login():
    return render_template('login.html')

@auth.route('/login', methods=['POST'])
def login_post():
    email = request.form.get('email')
    password = request.form.get('password')
    remember = True if request.form.get('remember') else False

    user = User.query.filter_by(email=email).first()
    if not user or not check_password_hash(user.password, password):
        flash('Invalid credentials')
        return redirect(url_for('auth.login'))

    login_user(user, remember=remember)
    return redirect(url_for('main.profile'))

@auth.route('/signup')
def signup():
    return render_template('signup.html')

@auth.route('/signup', methods=['POST'])
def signup_post():
    email = request.form.get('email')
    name = request.form.get('name')
    password = request.form.get('password')
    user = User.query.filter_by(email=email).first()
    if user:
        flash('Email address already exists')
        return redirect(url_for('auth.signup'))

    new_user = User(email=email, name=name, password=generate_password_hash(password, method='sha256'))
    db.session.add(new_user)
    db.session.commit()
    return redirect(url_for('auth.login'))

@auth.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('main.index'))


# 6. Protected Profile Route

from flask import Blueprint, render_template
from flask_login import login_required, current_user

main = Blueprint('main', __name__)

@main.route('/')
def index():
    return render_template('index.html')

@main.route('/profile')
@login_required
def profile():
    return render_template('profile.html', name=current_user.name)


'''
7. Example Login Template

{% extends "base.html" %}
{% block content %}
<form method="POST" action="/login">
    <input type="email" name="email" placeholder="Your Email" required>
    <input type="password" name="password" placeholder="Your Password" required>
    <label><input type="checkbox" name="remember"> Remember me</label>
    <button type="submit">Login</button>
</form>
{% endblock %}


8. Summary
Use Flask-Login for session management.

Store user credentials securely with password hashing.

Use the @login_required decorator to protect routes.

Organize your app with blueprints for modularity.


'''


In [ ]:
# 9. Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.

# 1. Setup
pip install flask flask_sqlalchemy

# 2. Basic Flask App Structure

from flask import Flask, request, jsonify, abort
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///books.db'
db = SQLAlchemy(app)

# Book Model
class Book(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(120), nullable=False)
    author = db.Column(db.String(120), nullable=False)
    description = db.Column(db.String(250))

# Initialize DB
@app.before_first_request
def create_tables():
    db.create_all()

# 3. CRUD Endpoints

@app.route('/books', methods=['POST'])
def create_book():
    data = request.get_json()
    new_book = Book(
        title=data['title'],
        author=data['author'],
        description=data.get('description', '')
    )
    db.session.add(new_book)
    db.session.commit()
    return jsonify({'id': new_book.id, 'message': 'Book created'}), 201


# Read All Books

@app.route('/books', methods=['GET'])
def get_books():
    books = Book.query.all()
    return jsonify([
        {'id': b.id, 'title': b.title, 'author': b.author, 'description': b.description}
        for b in books
    ])

# Read a Single Book

@app.route('/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    book = Book.query.get_or_404(book_id)
    return jsonify({'id': book.id, 'title': book.title, 'author': book.author, 'description': book.description})

# Update a Book

@app.route('/books/<int:book_id>', methods=['PUT'])
def update_book(book_id):
    book = Book.query.get_or_404(book_id)
    data = request.get_json()
    book.title = data.get('title', book.title)
    book.author = data.get('author', book.author)
    book.description = data.get('description', book.description)
    db.session.commit()
    return jsonify({'message': 'Book updated'})

# Delete a Book

@app.route('/books/<int:book_id>', methods=['DELETE'])
def delete_book(book_id):
    book = Book.query.get_or_404(book_id)
    db.session.delete(book)
    db.session.commit()
    return jsonify({'message': 'Book deleted'})

# 4. Run the App

if __name__ == '__main__':
    app.run(debug=True)




In [ ]:
# 10. Design a Flask app with proper error handling for 404 and 500 errors.

# Flask App with Custom 404 and 500 Error Pages

from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/cause-error')
def cause_error():
    # This will trigger a 500 Internal Server Error for demonstration
    raise Exception("This is a forced error!")

# Custom 404 Error Handler
@app.errorhandler(404)
def page_not_found(error):
    return render_template('404.html'), 404

# Custom 500 Error Handler
@app.errorhandler(500)
def internal_server_error(error):
    return render_template('500.html'), 500

if __name__ == '__main__':
    app.run(debug=True)

'''
#Templates
# templates/404.html

<!DOCTYPE html>
<html>
<head><title>404 Not Found</title></head>
<body>
    <h1>404 - Page Not Found</h1>
    <p>The page you are looking for does not exist.</p>
</body>
</html>


# templates/500.html

<!DOCTYPE html>
<html>
<head><title>500 Internal Server Error</title></head>
<body>
    <h1>500 - Internal Server Error</h1>
    <p>Oops! Something went wrong on the server. Please try again later.</p>
</body>
</html>

'''




In [1]:
''' Real-time Development: '''

' Real-time Development: '

In [ ]:
# 11. Create a real-time chat application using Flask-SocketIO.

# 1. Install Dependencies
pip install Flask Flask-SocketIO

# 2. Flask Backend (app.py)

from flask import Flask, render_template
from flask_socketio import SocketIO, emit

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app, cors_allowed_origins="*")

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('message')
def handle_message(data):
    # Broadcast the message to all clients
    emit('message', data, broadcast=True)

if __name__ == '__main__':
    socketio.run(app, debug=True)

'''
# 3. Frontend Template (templates/index.html)
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Flask-SocketIO Chat</title>
    <style>
        #messages { list-style-type: none; padding: 0; }
        .message { margin-bottom: 10px; }
    </style>
</head>
<body>
    <h2>Real-Time Chat</h2>
    <ul id="messages"></ul>
    <form id="form">
        <input id="input" autocomplete="off" placeholder="Type a message..." /><button>Send</button>
    </form>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.2.0/socket.io.js"></script>
    <script>
        var socket = io();
        var form = document.getElementById('form');
        var input = document.getElementById('input');
        var messages = document.getElementById('messages');

        form.addEventListener('submit', function(e) {
            e.preventDefault();
            if (input.value) {
                socket.emit('message', {message: input.value});
                input.value = '';
            }
        });

        socket.on('message', function(data) {
            var item = document.createElement('li');
            item.textContent = data.message;
            item.classList.add('message');
            messages.appendChild(item);
            window.scrollTo(0, document.body.scrollHeight);
        });
    </script>
</body>
</html>


4. How It Works
Backend: The Flask app serves the chat page and handles incoming messages via SocketIO. When a message is received, it is broadcast to all connected clients.
Frontend: Users type messages into a form. When submitted, the message is sent to the server via SocketIO and displayed for all users in real time.

5. Features and Extensibility
Real-time messaging: All clients see new messages instantly.
No page reloads: Communication is handled over WebSockets.

'''



In [ ]:
# 12. Build a Flask app that updates data in real-time using WebSocket connections.

# 1. Install Required Packages
pip install flask flask-socketio

# 2. Flask Application (app.py)

from flask import Flask, render_template
from flask_socketio import SocketIO, emit
import threading
import time

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

def background_thread():
    count = 0
    while True:
        time.sleep(1)  # Update every second
        count += 1
        # Broadcast the new count to all clients
        socketio.emit('update', {'data': count})

@socketio.on('connect')
def handle_connect():
    print('Client connected')

@socketio.on('disconnect')
def handle_disconnect():
    print('Client disconnected')

if __name__ == '__main__':
    # Start the background thread that sends updates
    thread = threading.Thread(target=background_thread)
    thread.daemon = True
    thread.start()
    socketio.run(app, debug=True)


'''
3. HTML Template (templates/index.html)

<!DOCTYPE html>
<html>
<head>
    <title>Real-Time Data Update</title>
</head>
<body>
    <h2>Real-Time Counter: <span id="counter">0</span></h2>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.2.0/socket.io.min.js"></script>
    <script>
        var socket = io();
        socket.on('update', function(msg) {
            document.getElementById('counter').textContent = msg.data;
        });
    </script>
</body>
</html>


How It Works
The Flask app serves a web page and manages WebSocket connections.
A background thread runs on the server, incrementing a counter every second and broadcasting the new value to all connected clients.
The client JavaScript listens for update events and updates the page instantly.

Extending Functionality
You can replace the counter with any real-time data (e.g., sensor values, notifications, chat messages).
For more complex data, send JSON objects and update multiple elements on the page.

'''


In [ ]:
# 13. Implement notifications in a Flask app using websockets to notify users of updates.

# 1. Install Dependencies
pip install flask flask-socketio

# 2. Flask Backend (app.py)
from flask import Flask, render_template
from flask_socketio import SocketIO, emit

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

# Example: Emit a notification to all clients
@app.route('/notify')
def notify():
    socketio.emit('notification', {'message': 'A new update is available!'}, broadcast=True)
    return 'Notification sent!'

# Optionally, notify user on connect
@socketio.on('connect')
def handle_connect():
    emit('notification', {'message': 'Welcome! You are connected.'})

if __name__ == '__main__':
    socketio.run(app, debug=True)

'''
3. Frontend (templates/index.html)

<!DOCTYPE html>
<html>
<head>
    <title>Flask-SocketIO Notifications</title>
</head>
<body>
    <h2>Real-Time Notifications</h2>
    <ul id="notifications"></ul>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.2.0/socket.io.min.js"></script>
    <script>
        var socket = io();
        var notifications = document.getElementById('notifications');
        socket.on('notification', function(data) {
            var item = document.createElement('li');
            item.textContent = data.message;
            notifications.appendChild(item);
        });
    </script>
</body>
</html>

How It Works
Server: When you access /notify, the server emits a notification event with a message to all connected clients using broadcast=True.
Client: The browser listens for the notification event and updates the notification list in real time.

Extending Functionality
User-specific notifications: Use SocketIO "rooms" to send notifications to specific users.
Custom events: Define and emit custom events for different notification types.
Persistent notifications: Store notifications in a database and fetch them on client connect.
'''